In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
schema = 'ds_data_for_complexity'
q = f"""show tables from dedicated.{schema}"""
%sql $q

stops_table_name = f'dedicated.{schema}.selected_stops_limited'
device_location_table_name = f'dedicated.{schema}.device_location_and_stops_subsampled'

query = f"""
with stops as(
    SELECT *
    FROM {stops_table_name}
),

events_ AS (
    SELECT *,
           LAG(stop_start_unix_timestamp) OVER (PARTITION BY lineage_id, data_provider, id_type ORDER BY zoned_datetime) AS prev_stop_start_unix_timestamp,
           LAG(stop_end_unix_timestamp) OVER (PARTITION BY lineage_id, data_provider, id_type ORDER BY zoned_datetime) AS prev_stop_end_unix_timestamp,
           LAG(zoned_datetime) OVER (PARTITION BY lineage_id, data_provider, id_type ORDER BY stop_start_unix_timestamp) AS prev_zoned_datetime,
           LAG(dwell_time_minutes) OVER (PARTITION BY lineage_id, data_provider, id_type ORDER BY stop_start_unix_timestamp) AS prev_dwell_time_minutes,
           LEAD(stop_start_unix_timestamp) OVER (PARTITION BY lineage_id, data_provider, id_type ORDER BY zoned_datetime) AS next_stop_start_unix_timestamp
    FROM stops
),

good_stops AS(
SELECT 
    lineage_id,  
    geohash,
    lat, lng,
    id_type,
    zoned_datetime, 
    COALESCE(stop_start_unix_timestamp, 0) as stop_start_unix_timestamp,
    COALESCE(stop_end_unix_timestamp, 0) as stop_end_unix_timestamp,
    COALESCE(prev_stop_start_unix_timestamp, 0) as prev_stop_start_unix_timestamp,
    COALESCE(prev_stop_end_unix_timestamp, 0) as prev_stop_end_unix_timestamp,
    CASE
        WHEN stop_start_unix_timestamp < prev_stop_end_unix_timestamp THEN 1 
        ELSE 0
    END as overlapping_stop,
    CAST(date_format(from_iso8601_timestamp(zoned_datetime), '%Y%m%d') AS INTEGER) AS event_date,
    data_provider
FROM events_
)

SELECT DISTINCT *
FROM good_stops
"""
df_stops = %sql $query

 * trino://localhost:9090/cuebiq/
Done.
 * trino://localhost:9090/cuebiq/
Done.


In [4]:
!pip install seaborn -q -q
!pip install geohash2 -q -q

import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
from scipy import stats
from keplergl import KeplerGl

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import ScalarFormatter

import geohash2
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [5]:
query = f"""
SELECT lineage_id, geohash_6
FROM {stops_table_name}
WHERE lineage_id % 1=0
"""

df = %sql $query

def convert_geohash6_to_geohash7(geohash6):
    lat, lon = geohash2.decode(geohash6)
    geohash7 = geohash2.encode(float(lat), float(lon), precision=7)
    return geohash7

# Convert geohash6 to geohash7
df['geohash_7'] = df['geohash_6'].apply(convert_geohash6_to_geohash7)

# Function to compute entropy
def compute_entropy(group):
    probabilities = group['lineage_id'].value_counts(normalize=True)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# Group by geohash_7 and compute entropy for each group
entropy_per_geohash = df.groupby('geohash_7').apply(compute_entropy).reset_index()
entropy_per_geohash.columns = ['geohash_7', 'entropy']


 * trino://localhost:9090/cuebiq/
Done.


In [6]:
print(entropy_per_geohash.head())

  geohash_7   entropy
0   dr4tvq4 -0.000000
1   dr4txe7  1.000000
2   dr4txt5  1.403065
3   dr4txwg  2.231120
4   dr4tz8g -0.000000


In [69]:
query = f"""
select *
from {device_location_table_name}
where event_date = 20240406
"""
df_dev_loc = %sql $query

 * trino://localhost:9090/cuebiq/
Done.


I used ChatGPT to convert the Python code to Trino SQL but I couldn't get it to work

In [8]:
query = f"""
WITH stops AS (
    SELECT lineage_id, geohash_6
    FROM {stops_table_name}
    WHERE lineage_id % 1 = 0
),

decoded AS (
    SELECT 
        lineage_id, 
        decode_geohash(geohash_6) AS (latitude, longitude)
    FROM stops
),

reencoded AS (
    SELECT 
        lineage_id, 
        encode_geohash(latitude, longitude, 7) AS geohash_7
    FROM decoded
),

entropy_calc AS (
    SELECT 
        geohash_7,
        -SUM(probability * LOG2(probability)) AS entropy
    FROM (
        SELECT 
            geohash_7,
            COUNT(lineage_id) * 1.0 / SUM(COUNT(lineage_id)) OVER (PARTITION BY geohash_7) AS probability
        FROM reencoded
        GROUP BY geohash_7, lineage_id
    ) probabilities
    GROUP BY geohash_7
)

SELECT geohash_7, entropy
FROM entropy_calc
ORDER BY entropy DESC
"""
df = %sql $query




 * trino://localhost:9090/cuebiq/


NameError: name 'DatabaseError' is not defined